In [93]:
import os
import sys
import json
sys.path.insert(0, os.path.abspath('/home/max/uva/mds_python'))

from mds.models import *
from mds.models.compact.user import UserCompactView
from mds.database.mongo import GetConfig
from mds.database.minio import GetMinioConfig

In [94]:
from mds.models.user import User
from mds.models.dataset import Dataset
from mds.models.download import Download
from mds.models.software import Software
from mds.models.computation import Computation

In [95]:
mongo_client = GetConfig()
mongo_database = mongo_client["test"]
mongo_collection = mongo_database["test_col"]

In [96]:
minio_client = GetMinioConfig()

In [97]:
test_user = User(**{
		"@id": "ark:99999/test-user1",
		"name": "Test User1",
		"type": "Person",
		"email": "testuser1@example.org",
		"password": "test1",
		"organizations": [],
		"projects": [],
		"datasets": [],
		"software": [],
		"computations": [],
		"evidencegraphs": []
		})

In [98]:
create_user_status = test_user.create(mongo_collection)

In [99]:
print(create_user_status)

Success: True	Message: 	StatusCode: 200


In [100]:
test_dataset = Dataset(**{
	"@id": "ark:99999/test-dataset",
	"@type": "evi:Dataset",
	"name": "test dataset",
	"owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		}
    })

In [101]:
create_dataset = test_dataset.create(mongo_collection)

In [102]:
print(str(create_dataset))

Success: True	Message: 	StatusCode: 201


In [103]:
test_dataset.id

'ark:99999/test-dataset'

In [104]:
test_data_download = Download(**{
	"@id": "ark:99999/test-download",
	"name": "test-data.csv",
	"@type": "DataDownload",
	"encodingFormat": ".txt",
	"encodesCreativeWork": test_dataset.id,
    })

In [105]:
with open("tests/test-data.csv", "rb") as file:
    create_data_download = test_data_download.register(file, mongo_collection, minio_client)

In [106]:
str(create_data_download)

'Success: True\tMessage: \tStatusCode: 201'

In [107]:
# upload software
test_software = Software(**{
    "@id": "ark:99999/test-software",
    "name": "script1",
    "@type": "Software",
    "owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		}
})

create_software_metadata = test_software.create(mongo_collection)

In [108]:
print(str(create_software_metadata))

Success: True	Message: 	StatusCode: 201


In [109]:
# upload script
test_software_object = Download(**{
    "@id": "ark:99999/test-software-script",
    "name": "test-script.py",
    "@type": "DataDownload",
    "owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		},
    "encodingFormat": ".py",
    "encodesCreativeWork": test_software.id
})

In [110]:
with open("tests/test-script.py", "rb") as file:
    print(test_software_object.register(file, mongo_collection, minio_client))

Success: True	Message: 	StatusCode: 201


In [111]:
test_computation  =  Computation(**{
        "@id": "ark:99999/test-comp",
        "name":"test computation",
        "owner": {
        "@id": "ark:99999/test-user1",
        "@type": "Person",
        "name": "Test User1",
        "email": "testuser1@example.org"
        },
        "container": "python3",
        "usedSoftware":test_software.id ,
        "usedDataset": test_dataset.id
})

In [112]:
print(test_computation.json(by_alias=True))

{"@id": "ark:99999/test-comp", "@type": "evi:Computation", "name": "test computation", "owner": {"@id": "ark:99999/test-user1", "@type": "Person", "name": "Test User1", "email": "testuser1@example.org"}, "dateCreated": null, "dateFinished": null, "associatedWith": [], "container": "python3", "usedSoftware": "ark:99999/test-software", "usedDataset": "ark:99999/test-dataset", "containerId": null, "@context": {"@vocab": "https://schema.org/", "evi": "https://w3id.org/EVI#"}}


In [113]:
print(test_computation.create(mongo_collection))

Success: True	Message: 	StatusCode: 201


## Testing Run Custom Container Code

In [114]:
import datetime
import pathlib

In [121]:
date_created = datetime.datetime.now()
script_id = test_computation.usedSoftware
dataset_id = test_computation.usedDataset

In [122]:
# find the locations of all the files
found_dataset = Dataset.construct(id=dataset_id)
read_dataset = found_dataset.read(mongo_collection)

In [123]:
found_dataset.json()

'{"id": "ark:99999/test-dataset", "type": "evi:Dataset", "distribution": [{"id": "ark:99999/test-download", "type": "DataDownload", "name": "test-data.csv", "contentUrl": "test dataset/test-data.csv"}], "context": {"@vocab": "https://schema.org/", "evi": "https://w3id.org/EVI#"}, "@id": "ark:99999/test-dataset", "@type": "evi:Dataset", "name": "test dataset", "owner": {"id": "ark:99999/test-user1", "type": "Person", "name": "Test User1", "email": "testuser1@example.org"}, "@context": {"@vocab": "https://schema.org/", "evi": "https://w3id.org/EVI#"}}'

In [45]:
print(read_dataset)

Success: True	Message: 	StatusCode: 200


In [126]:
dataset_files = []
dataset_files.append(found_dataset.distribution[0].contentUrl)

In [127]:
software = Software.construct(id=script_id)
print(software.read(mongo_collection))

Success: True	Message: 	StatusCode: 200


In [128]:
software.json()

'{"id": "ark:99999/test-software", "type": "evi:Software", "distribution": [{"id": "ark:99999/test-software-script", "type": "DataDownload", "name": "test-script.py", "contentUrl": "script1/test-script.py"}], "usedBy": [], "context": {"@vocab": "https://schema.org/", "evi": "https://w3id.org/EVI#"}, "@id": "ark:99999/test-software", "@type": "Software", "name": "script1", "owner": {"id": "ark:99999/test-user1", "type": "Person", "name": "Test User1", "email": "testuser1@example.org"}, "evi:usedBy": []}'

In [129]:
script_file = software.distribution[0].contentUrl

In [130]:
script_file

'script1/test-script.py'

In [131]:
# create a job-id string
test_computation.id

'ark:99999/test-comp'

In [132]:
# create a temporary landing folder for the output
temp = pathlib.Path(f"/tmp/{test_computation.id}")

In [134]:
# temp.mkdir(parents=True)

In [135]:
input_directory  = temp / "input"
software_directory = input_directory / "software"
data_directory = input_directory / "data"
output_directory = temp / "output"

software_directory.mkdir(parents=True)
data_directory.mkdir(parents=True)
output_directory.mkdir(parents=True)

#pathlib.Path("input")

In [136]:
pathlib.Path(script_file).name

'test-script.py'

In [137]:
# load software into software folder
script_filename = software_directory / pathlib.Path(script_file).name

minio_client.fget_object(
"test", script_file, str(script_filename)
)

In [138]:
# check that file was downloaded correctly
list(software_directory.glob("*"))

[PosixPath('/tmp/ark:99999/test-comp/input/software/test-script.py')]

In [139]:
for dataset in dataset_files:
    dataset_filename = data_directory / pathlib.Path(dataset).name
    minio_client.fget_object(
        "test", dataset, str(dataset_filename)
    )

In [140]:
list(data_directory.glob("*"))

[PosixPath('/tmp/ark:99999/test-comp/input/data/test-data.csv')]

### Execute Custom Container by mounting the temporary directory


In [86]:
# reset files

# remove all files
for dataset in data_directory.glob("*"):
    dataset.unlink()
    
for software in software_directory.glob("*"):
    software.unlink()
    
for output in output_directory.glob("*"):
    output.unlink()
    
# remove all folders
data_directory.rmdir()
software_directory.rmdir()
output_directory.rmdir()

# remove parent folder
input_directory.rmdir()

In [ ]:
# execute custom container

In [ ]:
test_computation.run_custom_container()

In [ ]:
register_computation(test_computation.containerId)